# How the garbage collector works

When developing a project in Python, the developer does not care about such things as allocating and freeing memory, which is typical for development in C ++ and C. As soon as the created objects are no longer needed, the garbage collector will automatically free the memory from under them.

CPython (the standard Python interpreter) uses two garbage collection mechanisms - object reference counting and the generational garbage collector (the gc module in the Python standard library).

### Reference counting algorithm

As you know, in Python all variables are object references. Naturally, several variables can refer to one object.

In [ ]:
typedef struct _object {
    _PyObject_HEAD_EXTRA
    Py_ssize_t ob_refcnt;
    PyTypeObject *ob_type;
} PyObject;

This is how the CPython structure looks like, on the basis of which other, more complex CPython primitives are implemented. Here, ob_refcnt is a variable that is incremented each time a reference is created to this object.

When that link goes down, naturally, the object's link count goes down.

Every time the reference count goes to zero, the object's destruction mechanism is triggered, and the references that this object had to other objects are also removed, so the destruction of one object can lead to a wave of deletions of other objects.

The main problem that the reference counting algorithm has is the inability to resolve circular dependencies. The generational garbage collector is designed to solve this problem.

In Python, using the sys.getrefcount() function, you can always find out the number of references to an object.

In [1]:
import sys
test_list = [1,2,3]
print(sys.getrefcount(test_list))
test_list_2 = test_list
print(sys.getrefcount(test_list_2))

2
3


In the example, a list is created and referenced by test_list, the reference count is 1. When sys.getrefcount() is used, the reference count rises to 2. Then we create another variable test_list_2 and assign test_list to it, the reference count is 2, and again when getrefcount() is applied, it becomes 3.

The sys.getrefcount() function usually returns one more reference count than expected.

This is due to the creation of a temporary reference to the argument that is passed to the function.

### Generational garbage collector (GC)

A circular reference is when one or more objects refer to each other.

In [2]:
a = []
a.append(a)

Here the list refers to itself.

In [3]:
a = {}
b = {}
a["b"] = b
b["a"] = a

Here the dictionaries refer to each other.

If you call the del() method, then references to objects will be deleted. If there were no GC, then the objects would still hang in memory, although they would be inaccessible to the developer.

Unlike reference counting, GC does not fire in real time and runs periodically. Built-in heuristics are used to determine the launch frequency.

Generational means "relating to a particular generation" and, indeed, GC divides all objects into three generations. Initially, all objects are placed in the first generation, live there for a while and most of them are cleared, the rest is moved to the second generation and then to the third. The higher the generation, the less often it is scanned for garbage.

To detect circular references, the GC iterates over each object in a generation and temporarily removes all references to which that object refers. After a complete crawl, all objects with a reference count less than two are considered unreachable and are removed.

GC as a module gives the developer the ability to control the garbage collector for circular references.

In [5]:
import gc
gc.enable() # enable the garbage collector
gc.disable() # Disable the garbage collector
gc.collect(generation=2) # Explicitly initiates a garbage collector pass before it starts automatically

43

### Memory management

Python is a dynamic language, during which many objects are created. Some of them may contain other objects - lists, dictionaries, tuples.

If a newly created object takes up more than 512 bytes in memory, then Python sends it to the standard C allocator.

To optimize memory allocation for small objects less than 512 bytes in size, Python allocates large blocks of memory in advance, which are divided into three levels - arenas, pools, blocks.

A block is a piece of memory of a certain size, each block contains only one Python object of a fixed size, a multiple of 8. The block size is from 8 to 512 bytes.

A pool is a collection of blocks of the same size. Typically, the pool size is 4 KB. If an object is destroyed, then the memory it occupies is not returned to the operating system, but is saved for future objects of the same size.

The pool and blocks do not allocate memory directly, but use the memory allocated by arenas.

The arena is a 256 KB chunk of memory allocated from the operating system heap, which holds 64 pools.

Schematically, the arena looks like this:

In [ ]:
            Arena
Pool(4kB)	Pool(4kB)	    Pool(4kB)
Pool(4kB)	Pool(4kB)	    Pool(4kB)
Pool(4kB)	FreePool(4kB)	FreePool(4kB)
...	        ...	            ...

Due to the arrangement of the memory manager in Python, the release of operating system resources rarely occurs in practice. This is only possible if the arena contains 64 empty pools. For short operations, when you need to quickly perform calculations, this is possible. With long-lived processes, you can observe a large consumption of memory, which, in fact, is not used efficiently.